## 0日目

### パラメータ多相

In [4]:
def head[A](xs: List[A]): A = xs(0)

defined function head

In [5]:
head(1 :: 2 :: Nil)

res4: Int = 1

In [6]:
case class Car(make: String)

defined class Car

In [7]:
head(Car("Civic") :: Car("CR-V") :: Nil)

res6: Car = Car("Civic")

* parametric polymorphism
* Aが何であっても動作する

### 派生型多態

In [8]:
def plus[A](a1: A, a2: A): A = ???

defined function plus

In [9]:
trait Plus[A] {
    def plus(a2: A): A
}

defined trait Plus

In [10]:
def plus[A <: Plus[A]](a1: A, a2: A): A = a1.plus(a2)

defined function plus

* 派生型 subtype
* `<:` : AはBのサブ型(Javaでいう <A extends Plus>)
* Javaでいう継承もしくは実装していれば使えるよって感じ

### アドホック多相

In [11]:
trait Plus[A] {
    def plus(a1: A, a2: A): A
}

defined trait Plus

In [12]:
def plus[A: Plus](a1: A, a2: A): A = implicitly[Plus[A]].plus(a1, a2)

defined function plus

* 現時点では、いまいち派生型多態との違いがわからない

### sum関数

In [13]:
def sum(xs: List[Int]): Int = xs.foldLeft(0) { _ + _ }

defined function sum

In [14]:
sum(List(1, 2, 3, 4))

res13: Int = 10

### Monoid

In [15]:
object IntMonoid {
    def mappend(a: Int, b: Int): Int = a + b
    def mzero: Int = 0
}

defined object IntMonoid

In [25]:
def sum(xs: List[Int]): Int = xs.foldLeft(IntMonoid.mzero)(IntMonoid.mappend)
sum(List(1, 2, 3, 4))

defined function sum
res24_1: Int = 10

* `0` って書いていたのを `mzero` に抽象化できた

In [26]:
trait Monoid[A] {
    def mappend(a1: A, a2: A): A
    def mzero: A
}

object IntMonoid extends Monoid[Int] {
    def mappend(a: Int, b: Int): Int = a + b
    def mzero: Int = 0
}

defined trait Monoid
defined object IntMonoid

In [27]:
def sum(xs: List[Int], m: Monoid[Int]): Int = xs.foldLeft(m.mzero)(m.mappend)
sum(List(1, 2, 3, 4), IntMonoid)

defined function sum
res26_1: Int = 10

In [28]:
def sum[A](xs: List[A])(implicit m: Monoid[A]): A = xs.foldLeft(m.mzero)(m.mappend)

implicit val intMonoid = IntMonoid
sum(List(1, 2, 3, 4))

defined function sum
intMonoid: IntMonoid.type = $sess.cmd25Wrapper$Helper$IntMonoid$@48a7cc7b
res27_2: Int = 10

In [29]:
def sum[A: Monoid](xs: List[A]): A = {
    val m = implicitly[Monoid[A]]
    xs.foldLeft(m.mzero)(m.mappend)
}

sum(List(1, 2, 3, 4))

defined function sum
res28_1: Int = 10

* implicit の使い道がよくわかる例

In [30]:
trait Monoid[A] {
    def mappend(a1: A, a2: A): A
    def mzero: A
}
object Monoid {
    implicit val IntMonoid: Monoid[Int] = new Monoid[Int] {
        def mappend(a: Int, b: Int): Int = a + b
        def mzero: Int = 0
    }
    implicit val StringMonoid: Monoid[String] = new Monoid[String] {
        def mappend(a: String, b: String): String = a + b
        def mzero: String = ""
    }
}
def sum[A: Monoid](xs: List[A]): A = {
    val m = implicitly[Monoid[A]]
    xs.foldLeft(m.mzero)(m.mappend)
}

defined trait Monoid
defined object Monoid
defined function sum

In [31]:
sum(List("a", "b", "c"))
sum(List(1, 2, 3, 4))

res30_0: String = "abc"
res30_1: Int = 10

In [32]:
val multiMonoid: Monoid[Int] = new Monoid[Int] {
    def mappend(a: Int, b: Int): Int = a * b
    def mzero: Int = 1
}

sum(List(1, 2, 3, 4))(multiMonoid)

multiMonoid: Monoid[Int] = $sess.cmd31Wrapper$Helper$$anon$1@2b133b72
res31_1: Int = 24

### FoldLeft

In [33]:
object FoldLeftList {
    def foldLeft[A, B](xs: List[A], b: B, f: (B, A) => B) = xs.foldLeft(b)(f)
}

def sum[A: Monoid](xs: List[A]): A = {
    val m = implicitly[Monoid[A]]
    FoldLeftList.foldLeft(xs, m.mzero, m.mappend)
}

defined object FoldLeftList
defined function sum

In [34]:
sum(List(1, 2, 3, 4))

sum(List("a", "b", "c"))

sum(List(1, 2, 3, 4))(multiMonoid)

res33_0: Int = 10
res33_1: String = "abc"
res33_2: Int = 24

In [35]:
trait FoldLeft[F[_]] {
    def foldLeft[A, B](xs: F[A], b: B, f: (B, A) => B): B
}
object FoldLeft {
    implicit val FoldLeftList: FoldLeft[List] = new FoldLeft[List] {
        def foldLeft[A, B](xs: List[A], b: B, f: (B, A) => B) = xs.foldLeft(b)(f)
    }
}

def sum[M[_]: FoldLeft, A: Monoid](xs: M[A]): A = {
    val m = implicitly[Monoid[A]]
    val f1 = implicitly[FoldLeft[M]]
    f1.foldLeft(xs, m.mzero, m.mappend)
}

defined trait FoldLeft
defined object FoldLeft
defined function sum

In [37]:
sum(List(1, 2, 3, 4))

sum(List("a", "b", "c"))

res36_0: Int = 10
res36_1: String = "abc"

### メソッド注入
* enrich my libraryパターン

In [38]:
def plus[A: Monoid](a: A, b: A): A = implicitly[Monoid[A]].mappend(a, b)

plus(3, 4)

defined function plus
res37_1: Int = 7

In [39]:
trait MonoidOp[A] {
    val F: Monoid[A]
    val value: A
    def |+|(a2: A) = F.mappend(value, a2)
}

implicit def toMonoidOp[A: Monoid](a: A): MonoidOp[A] = new MonoidOp[A] {
    val F = implicitly[Monoid[A]]
    val value = a
}

defined trait MonoidOp
defined function toMonoidOp

In [43]:
3 |+| 4
"a" |+| "b"

res42_0: Int = 7
res42_1: String = "ab"

* implicit conversions を使った例なのかな。実際は次のコードが実行されている気がする

In [45]:
toMonoidOp(3).|+|(4)
toMonoidOp("a").|+|("b")

res44_0: Int = 7
res44_1: String = "ab"

### 標準型構文

In [48]:
import scalaz._, Scalaz._

1.some | 2

import scalaz._, Scalaz._


res47_1: Int = 1

In [46]:
Some(1).getOrElse(2)

res45: Int = 1

In [49]:
(1 > 10)? 1 | 2

res48: Int = 2

In [50]:
if (1 > 10) 1 else 2

res49: Int = 2